In [52]:
%%capture
!pip install -U pystac-client
from pystac_client import Client
import numpy as np

In [53]:
#list of the dataset collections ingested in Stac - provided by aime in the github ticket.

candidatedata_list = ['Sentinel-1A Dual-pol ground projected high and full resolution images',
                      'Sentinel-1A slant-range product',
                      'ESACCI_Biomass_L4_AGB_V3_100m_2010',
                      'ESACCI_Biomass_L4_AGB_V3_100m_2017',
                      'ESACCI_Biomass_L4_AGB_V3_100m_2018',
                      'Global Ecosystem Dynamics Investigation (GEDI) Calibration/Validation Airborne Lidar Dataset',
                      'Global Ecosystem Dynamics Investigation (GEDI) Calibration/Validation Airborne Lidar Dataset (Compressed)',
                      'Global Ecosystem Dynamics Investigation (GEDI) Calibration/Validation Field Survey Dataset v2',
                      'GlobCover Global Land Cover Product (2005-06)',
                      'GlobCover Global Land Cover Product (2009)',
                      'Sentinel-1B Dual-pol ground projected high and full resolution images, Texas/Mexico and Peru, September 2016-July 2019',
                      'AfriSAR UAVSAR Geocoded Covariance Matrix product Generated Using NISAR Tools',
                      'Advance Land Observing Satellite Phased Array type L-band Synthetic Aperture Radar Radiometric Terrain-Corrected High Resolution products, Equatorial Western Africa, May 2006-March 2011',
                      'AfriSAR UAVSAR Geocoded SLCs Generated Using NISAR Tools',
                      'AfriSAR UAVSAR Normalization Area Generated Using NISAR Tools',
                      'AfriSAR UAVSAR Ungeocoded Covariance Matrix product Generated Using NISAR Tools',
                      'AfriSAR UAVSAR Vertical Wavenumber (KZ) Generated Using NISAR Tools',
                      'AFRISAR_DLR',
                      'AFRISAR_DLR2',
                      'Global 25m Resolution PALSAR-2/PALSAR Mosaic',
                      'Global Forest Change 2000-2017',
                      'Arctic-Boreal Vulnerability Experiment Uninhabited Aerial Vehicle Synthetic Aperture Radar Polarimetric SAR',
                      'Global 25m Resolution PALSAR-2/PALSAR Forest/Non-Forest Map',
                      'LVIS Facility L1B Geolocated Return Energy Waveforms V001',
                      'ABoVE LVIS L1B Geolocated Return Energy Waveforms V001',
                      'AfriSAR LVIS L2 Geolocated Surface Elevation Product V001',
                      'AfriSAR UAVSAR Coregistered SLCs Generated Using NISAR Tools',
                      'AfriSAR: Aboveground Biomass for Lope, Mabounie, Mondah, and Rabi Sites,Gabon',
                      'BIOSAR1',
                      'Landsat 8 Operational Land Imager (OLI) Surface Reflectance Analysis Ready Data (ARD) V1, Peru and Equatorial Western Africa, April 2013-January 2020']

print(len(candidatedata_list))

30


In [54]:
# STAC API root URL
URL = 'https://stac.maap-project.org/'
cat = Client.open(URL)
cat

ID: stac-fastapi
Title: maap-stac
Description: maap-stac
type: Catalog
"conformsTo: ['http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter', 'https://api.stacspec.org/v1.0.0-rc.1/ogcapi-features', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#sort', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#query', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#context', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:basic-cql', 'https://api.stacspec.org/v1.0.0-rc.1/core', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:cql-text', 'https://api.stacspec.org/v1.0.0-rc.1/collections', 'https://api.stacspec.org/v1.0.0-rc.1/item-search', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#fields']"
https://raw.githubusercontent.com/radiantearth/stac-api-spec/v1.0.0-rc.1/fragments/context/json-schema/schema.json
ID: GlobCover_09
Title: GlobCover Global Land Cover Product (2009)
"Description: GlobCover is an European Space Agency (ESA) initiative which began in 2005 in partnership with several organizations including JRC, EEA, FAO, UNEP, GOFC-GOLD and IGBP. The aim of the project was to develop a service capable of delivering global composites and land cover maps using as input observations from the 300m MERIS sensor on board the ENVISAT satellite mission. The GlobCover land cover maps cover 2 periods: December 2004 - June 2006 and January - December 2009. This dataset provides the January - December 2009 data."
type: Collection
title: GlobCover Global Land Cover Product (2009)


In [74]:
#finding the difference between collections in Candidate data list and collections in Stac
collectionid_list = []

#getting all collections from Stac
for collection in cat.get_all_collections():                                  
    print(collection.title)
    collectionid_list.append(collection.title)

#Compute the difference between collections in Candidate data list and collections in Stac
diff = list(set(candidatedata_list)-set(collectionid_list))
print(diff)


GlobCover Global Land Cover Product (2009)
AfriSAR UAVSAR Coregistered SLCs Generated Using NISAR Tools
BIOSAR1
AfriSAR: Aboveground Biomass for Lope, Mabounie, Mondah, and Rabi Sites, Gabon
Landsat 8 Operational Land Imager (OLI) Surface Reflectance Analysis Ready Data (ARD) V1, Peru and Equatorial Western Africa, April 2013-January 2020
ABoVE LVIS L1B Geolocated Return Energy Waveforms V001
AfriSAR UAVSAR Ungeocoded Covariance Matrix product Generated Using NISAR Tools
GlobCover Global Land Cover Product (2005-06)
Global 25m Resolution PALSAR-2/PALSAR Forest/Non-Forest Map
GEDI L2A Elevation and Height Metrics Data Global Footprint Level V002
AfriSAR LVIS L2 Geolocated Surface Elevation Product V001
AFRISAR_DLR
Gridded Boreal Aboveground Biomass Density c.2020 at 30m resolution
AfriSAR UAVSAR Normalization Area Generated Using NISAR Tools
Global Ecosystem Dynamics Investigation (GEDI) Calibration/Validation Airborne Lidar Dataset (Compressed)
Global Ecosystem Dynamics Investigation (

In [34]:
from maap.maap import MAAP

maap = MAAP(maap_host="api.ops.maap-project.org")
username = maap.profile.account_info()["username"]
print(username)

susan121212


In [110]:
#getting collections from CMR to find the Granule length

cmrshortname = []
stacid = []
conceptid_name = {}
cmrgranule_len={}
results = maap.searchCollection(cmr_host="cmr.maap-project.org")
test = []

excl_list =["C1200109550-ESA_MAAP",'C1200231030-NASA_MAAP']              #excluding these collections as the result field was empty
for collection in results:
    conceptid = collection['concept-id']
    if conceptid in excl_list:
        continue
    conceptid_name[conceptid] = collection['Collection']['ShortName']    #getting COnceptID for the corresponding shortname for each collection from CMR

for collection in cat.get_all_collections():                             #getting all collections from Stac
    stacid.append(collection.id)

for concid,name in conceptid_name.items():                               #using ConceptID for each collection and retrieving the Granule length
    if name in stacid:
        results = maap.searchGranule(cmr_host='cmr.maap-project.org',concept_id=concid, limit=100000)
        cmrgranule_len[name]=len(results)
print(cmrgranule_len)

C1200110748-NASA_MAAP;ABLVIS1B
C1200125288-NASA_MAAP;ABLVIS2
C1200120106-NASA_MAAP;ALOS_PSR_RTC_HIGH
C1200116820-NASA_MAAP;ALOS_PSR_L1.5
C1200110728-NASA_MAAP;AFLVIS1B
C1200110729-NASA_MAAP;AFLVIS2
C1200000308-NASA_MAAP;AfriSAR_UAVSAR_Coreg_SLC
C1200109238-NASA_MAAP;AfriSAR_UAVSAR_Geocoded_Covariance
C1200109237-NASA_MAAP;AfriSAR_UAVSAR_Geocoded_SLC
C1200109244-NASA_MAAP;AfriSAR_UAVSAR_Normalization_Area
C1200109245-NASA_MAAP;AfriSAR_UAVSAR_Ungeocoded_Covariance
C1200109243-NASA_MAAP;AfriSAR_UAVSAR_KZ
C1200125308-NASA_MAAP;AfriSAR_UAVSAR_MLC
C1200125309-NASA_MAAP;AfriSAR_UAVSAR_SLC
C1200115768-NASA_MAAP;AfriSAR_AGB_Maps_1681
C1200097474-NASA_MAAP;AfriSAR_LVIS_Footprint_Cover_1591
C1200000521-NASA_MAAP;Polarimetric_CT_1601
C1202028168-NASA_MAAP;Afrisar_LVIS_Biomass_VProfiles_1775
C1200000492-NASA_MAAP;AfriSAR_Mondah_Field_Data_1580
C1200015148-NASA_MAAP;Polarimetric_height_profile_1577
C1200015068-NASA_MAAP;PolInSAR_Canopy_Height_1589
C1200109552-ESA_MAAP;AFRISAR_DLR
C1200015069-ESA_MAA

In [9]:
#Getting the Item length for each collection in Stac
ex_list = ["GEDI_CalVal_Lidar_Data_Compressed"]

for collection in cat.get_all_collections():
    if collection.id in ex_list:
        continue
    items = collection.get_items()
    item_count = len([item for item in items])
    stac_collection_dict[collection.id] = item_count
print(stac_collection_dict)

NameError: name 'cat' is not defined

In [ ]:
# Checking if cmr and stac have same number of items/granules.

for stac_name, num_items in stac_collection.items():
    if stac_name in cmrgranule_len.keys():
        if cmrgranule_len[stac_name] == num_items:
            print(f"{stac_name} has equal number of items/granules")
        else:
            print(f"{stac_name} has unequal number of items/granules.")